<a href="https://colab.research.google.com/github/aarshitaacharya/peft-techniques/blob/main/R13_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install transformers peft datasets dash pyngrok

# Import required libraries
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from pyngrok import ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 747.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [ ]:
# Load dataset
dataset = load_dataset("THUDM/humaneval-x", "js")

# Model configuration
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side='right'
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

humaneval-x.py:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

js/test/0000.parquet:   0%|          | 0.00/106k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
# Tokenization
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples['prompt'], padding="max_length", truncation=True, max_length=512
    )
    tokenized_inputs['labels'] = tokenized_inputs['input_ids']
    # Ignore padding in loss calculation by setting it to -100
    tokenized_inputs['labels'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels]
        for labels in tokenized_inputs['labels']
    ]
    return tokenized_inputs

tokenized_dataset = dataset['test'].map(tokenize_function, batched=True)

# Split into train and eval sets
train_size = int(0.8 * len(tokenized_dataset))
train_indices = random.sample(range(len(tokenized_dataset)), train_size)
train_dataset = tokenized_dataset.select(train_indices)
eval_indices = list(set(range(len(tokenized_dataset))) - set(train_indices))
eval_dataset = tokenized_dataset.select(eval_indices)

# Format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query_key_value"],
)
lora_model = get_peft_model(model, lora_config)

# Training setup
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./lora_model",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
    logging_dir="./logs",  # For detailed logging
    logging_steps=100,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

#56266cf7350cb31473487d39b3399ea8f20adfd3

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=40, training_loss=2.538662338256836, metrics={'train_runtime': 97.9385, 'train_samples_per_second': 6.688, 'train_steps_per_second': 0.408, 'total_flos': 591425905360896.0, 'train_loss': 2.538662338256836, 'epoch': 4.848484848484849})

In [ ]:
# Inference Mode
lora_model.eval()

# Web App
app = Dash(__name__)

app.layout = html.Div(style={'backgroundColor': '#f8f9fa', 'padding': '20px'}, children=[
    html.H1("Model Inference Web App", style={'textAlign': 'center', 'color': '#343a40'}),
    dcc.Input(
        id='input-text', type='text', placeholder='Enter input text here...',
        style={'width': '100%', 'padding': '10px', 'fontSize': '18px', 'marginBottom': '10px'}
    ),
    html.Button(
        'Submit', id='submit-button', n_clicks=0,
        style={'backgroundColor': '#007bff', 'color': 'white', 'padding': '10px 20px',
               'border': 'none', 'borderRadius': '5px', 'cursor': 'pointer'}
    ),
    html.Div(
        id='output-prediction', style={'marginTop': '20px', 'fontSize': '18px',
                                       'color': '#495057', 'border': '1px solid #ced4da',
                                       'padding': '10px', 'borderRadius': '5px'}
    ),
])

@app.callback(
    Output('output-prediction', 'children'),
    [Input('submit-button', 'n_clicks')],
    [Input('input-text', 'value')]
)
def update_output(n_clicks, input_text):
    if n_clicks > 0 and input_text:
        # Use generate for meaningful predictions
        prompt = f"// Generate JavaScript code for:\n{input_text}\n"
        inputs = tokenizer(prompt, return_tensors='pt').to(lora_model.device)

        with torch.no_grad():
            outputs = lora_model.generate(
                **inputs,
                max_length=150,
                temperature=0.7,  # Adjust for creativity
                top_p=0.95,       # Nucleus sampling
                num_return_sequences=1
            )

        decoded_prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return html.Pre(f"Generated Code:\n{decoded_prediction}", style={'whiteSpace': 'pre-wrap'})

    return "Enter a prompt and click Submit."

In [ ]:
# Run the App
ngrok.set_auth_token('2nfGHPST8eqZs7wQpzzzFx9E6nV_7YoWjiP1jzEsZ83KypzDB')  # Replace with your ngrok token
public_url = ngrok.connect(8050)
print(f"Public URL: {public_url}")

app.run_server(port=8050)

Public URL: NgrokTunnel: "https://49bc-34-105-119-85.ngrok-free.app" -> "http://localhost:8050"


<IPython.core.display.Javascript object>